# Sentiment Analysis with Deep Learning using BERT

This is a project delievered by Coursera

- Exploratory data analysis and preprocessing

- Training/validation split

- Loading tokenizer and encoding our data

- setting up BERT pretrained model

- Creating data loaders

- setting up optimizer and scheduler

- Definingour performance metrics

- Creating training loop

- Loading and evaluating models

### Exploratory data analysis

In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm #progress bar 

In [3]:
df = pd.read_csv('C:/Anaconda/Jupyter Notebook/Data/smile-annotations-final.csv',names=['id','text','category'])
df.set_index('id',inplace=True)
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [4]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [5]:
# remove multiple emotions and nocode
df = df[~df.category.str.contains('\|')] #tilde just to exclude
df = df[df.category != 'nocode']

In [6]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [7]:
df.head()

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [8]:
possible_labels = df.category.unique()
possible_labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [9]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [10]:
# now create a new column with the index label
df['label'] = df.category.replace(label_dict)

In [11]:
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


### Train/Test split

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_val,y_train,y_val = train_test_split(
    df.index.values, # x-values ID
    df.label.values, # y-values label
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values # split across each label because the dataset is imbalance
)

In [14]:
#create a data_type column and set initial values
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [15]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_val,'data_type'] = 'val'

In [16]:
# Check how data splited across different label
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

### Loading tokenizer and encoding data

In [17]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True                                     
) 
#tokenizer.tokenize splits the sequence into tokens available in the tokenizer vocabulary

In [20]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True, #bound the maxi words of sentences
    max_length = 256,
    return_tensors = 'pt'
)
#the tokenizer will return output a dictionary with all the arguments necessary for its corresponding model to work properly

#identical way to get validation set
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt' #pytorch
)

input_ids_train = encoded_data_train['input_ids'] 
# convert the sequence to numerical indices
attention_masks_train = encoded_data_train['attention_mask'] 
# attention mask indicates to the model which tokens should be attended to and which should not
# it returns a binary tensor sequence indicating the position of the padded indices so that the model does not attend to
# for example, when 2 sentences have differnet length, we have use attention mask to pad the short one up to the length of the long sentence,
# and the padded value is 0 
labels_train = torch.tensor(df[df.data_type=='train'].label.values)


input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\yuanp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
# now we have encoded dataset
dataset_train =  TensorDataset(
    input_ids_train,
    attention_masks_train,
    labels_train
)

dataset_val =  TensorDataset(
    input_ids_val,
    attention_masks_val,
    labels_val
)

In [22]:
#check the datasets 
len(dataset_train) / (len(dataset_train)+len(dataset_val))

0.849426063470628

### Setting up BERT pretrained model

In [23]:
from transformers import BertForSequenceClassification

In [24]:
#We will treat each tweet as an unique sequence
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Creating data loaders

In [25]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

In [26]:
batch_size = 3

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

### Setting up optimiser

In [27]:
from transformers import AdamW,get_linear_schedule_with_warmup

In [28]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
) 

In [29]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

### Defining performance metrics

In [30]:
import numpy as np
from sklearn.metrics import f1_score

In [31]:
def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average='weighted')


In [32]:
def accuracy_per_class(preds,labels):
    labels_dict_inverse = {v:k for k,v in label_dict.items()}
    
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {labels_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

### Creating training loop


In [33]:
import random

seed_val =17
random.seed (seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [35]:
def evaluate(dataloader_val):
    
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],'attention_mask': batch[1],'labels':batch[2]}

        with torch.no_grad(): #ignoring the gradient and disable them
            outputs = model(**inputs)
        
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy() #in case using GPUs, detach will pull out the values onto CPUs in order to use them in numpy
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1,epochs+1)):
    
    model.train() #set to training mode
    
    loss_train_total = 0 #initialize the loss at 0 and then add up the loss of all batches when training
    progress_bar = tqdm(dataloader_train,desc = 'Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False) #  see the training progress to better control when training is long 
    for batch in progress_bar:
        
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids'      : batch[0],
            'attention_mask' : batch[1],
            'labels'         : batch[2]
        }
        
        outputs = model(**inputs) #** will unpack the input dictionary
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss':':.3f'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(),f'BERT Models/BERT_ft_epoch{epoch}.model') #save the model per epoch
    # state_dict is a Python dictionary object that maps each layer to its parameter tensor
    
    tqdm.write(f"\nEpoch: {epoch}")
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss,predictions,true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss:{val_loss}')
    tqdm.write(f'F1 Score(weighted):{val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/420 [00:00<?, ?it/s]


Epoch: 1
Training loss: 0.8541418302072478
Validation loss:0.747589447312057
F1 Score(weighted):0.7616996426485795


Epoch 2:   0%|          | 0/420 [00:00<?, ?it/s]


Epoch: 2
Training loss: 0.5353040979149574
Validation loss:0.7767251955438406
F1 Score(weighted):0.8097974960939878


...


Epoch: 8
Training loss: 0.07770299998186861
Validation loss:0.9014391776088936
F1 Score(weighted):0.8497265586685274


Epoch 9:   0%|          | 0/420 [00:00<?, ?it/s]


Epoch: 9
Training loss: 0.04789661335553752
Validation loss:0.8947619981695122
F1 Score(weighted):0.8536955153473312


Epoch 10:   0%|          | 0/420 [00:00<?, ?it/s]


Epoch: 10
Training loss: 0.03927712768878943
Validation loss:0.9500151255730694
F1 Score(weighted):0.8460602118104921


In [ ]:
torch.cuda.empty_cache()

### Evaluating model

In [36]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [37]:
model.to(device)
pass

In [38]:
#load the saved model
model.load_state_dict(
    torch.load('BERT Models/BERT_ft_epoch10.model',
    map_location = torch.device('cpu')) #map the gpu trained model onto cpu
)

<All keys matched successfully>

In [39]:
_, predictions,true_vals = evaluate(dataloader_val)

  0%|          | 0/75 [00:00<?, ?it/s]

In [42]:
np.argmax(predictions,axis=1).flatten()

array([0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 5, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 4, 0, 2, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 5, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 5, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [41]:
true_vals

array([0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 4, 0,
       0, 0, 0, 0, 0, 3, 0, 5, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 2, 0, 0, 5, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0,
       0, 1, 0], dtype=int64)

In [ ]:
accuracy_per_class(predictions,true_vals)

Class: happy
Accuracy:161/171

Class: not-relevant
Accuracy:19/32

Class: angry
Accuracy:7/9

Class: disgust
Accuracy:0/1

Class: sad
Accuracy:1/5

Class: surprise
Accuracy:2/5

